In [ ]:
!pip install -U -q langchain
!pip install -U -q langchain_google_genai
!pip install -U -q langchain_community
!pip install -U -q langchain_text_splitters
!pip install -U -q langchain_core
!pip install -U -q langchainhub
!pip install -U -q bs4
!pip install -U -q duckduckgo-search
!pip install -U -q wikipedia
!pip install -U -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 

**LANGCHAIN com Google Gemini**

Nesta etapa inicial será feita a importação das bibliotecas, configuraçao da API KEY e definição do modelo e do embbeding que será utilizado

In [ ]:
import os
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
import bs4

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate

from langchain_core.runnables import RunnableSequence

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("insert_google_key_here")

llm = ChatGoogleGenerativeAI(model="gemini-pro")
google_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

**Query informada pelo usuário**

In [ ]:
query= """
Vou viajar para Londres em agosto de 2024.
Quero que faça para um roteiro de viagem para mim com eventos que irão ocorrer na data da viagem e com o preço de passagem de São Paulo para Londres.
"""

** *texto em itálico*Funções auxiliares**

Foram criadas funções para consultar o DuckDuckGo e a Wikipedia, e também para fazer o parse de um site com dicas sobre o pais que vai ser visitado

Documentação:

https://python.langchain.com/v0.2/docs/integrations/document_loaders/web_base/

https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html

https://smith.langchain.com/hub

https://smith.langchain.com/hub/hwchase17/react

https://python.langchain.com/v0.2/docs/integrations/tools/

In [ ]:
def researchAgent(query, llm):
  tools = load_tools(["ddg-search", "wikipedia"], llm=llm)
  prompt = hub.pull("hwchase17/react")
  agent = create_react_agent(llm, tools, prompt)
  agent_executor = AgentExecutor(agent=agent, tools=tools, prompt=prompt, verbose=True)
  webContext = agent_executor.invoke({ "input": query })
  return webContext['output']

# print(researchAgent(query, llm))

def loadData():
  loader = WebBaseLoader(
  web_paths= ("https://www.dicasdeviagem.com/inglaterra/",),
  bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("postcontentwrap", "pagetitleloading background-imaged loading-dark"))),)
  docs = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  splits = text_splitter.split_documents(docs)
  vectorstore = Chroma.from_documents(documents=splits, embedding=google_embeddings)
  retriever = vectorstore.as_retriever()
  return retriever


**Pesquisa de documentos vetorizados**

Utilizando o conteudo do site que foi vetorizado e armazendo de forma local no banco de vetores Chroma, será feita uma pesquisa por semelhança com base na consulta feita pelo usuário

In [ ]:
def getRelevantDocs(query):
  retriever = loadData()
  relevant_documents = retriever.invoke(query)
  return relevant_documents


**Supervisor**

Para integrar tanto a busca na Web e a busca por documentos, foi criado um supervisor para integrar todos os conteúdos numa única consulta.

In [ ]:
def supervisorAgent(query, llm, webContext, relevant_documents):
  prompt_template = """
  Você é um gerente de uma agência de viagens. Sua resposta final deverá ser um roteiro de viagem completo e detalhado.
  Utilize o contexto de eventos e preços de passagens, o input do usuário e também os documentos relevantes para elaborar o roteiro.
  Contexto: {webContext}
  Documento relevante: {relevant_documents}
  Usuário: {query}
  Assistente:
  """

  prompt = PromptTemplate(
    input_variables=['webContext', 'relevant_documents', 'query'],
    template = prompt_template
  )

  sequence = RunnableSequence(prompt | llm)
  response = sequence.invoke({"webContext": webContext, "relevant_documents": relevant_documents, "query": query})
  return response

Execução

Para executar o nosso supervisor foi criada uma função que recebe como parâmetros a consulta do usuário e o nosso LLM que é o LangChain

In [ ]:
def getResponse(query, llm):
  webContext = researchAgent(query, llm)
  relevant_documents = getRelevantDocs(query)
  response = supervisorAgent(query, llm, webContext, relevant_documents)
  return response

print(getResponse(query, llm).content)



> Entering new AgentExecutor chain...
I need to find out what events are happening in London in August 2024 and the price of a plane ticket from São Paulo to London.
Action: duckduckgo_search
Action Input: events in London August 2024Unmissable events in London in August. Celebrate August bank holiday weekend (24-26 Aug) at Europe's largest street festival, as the iconic Notting Hill Carnival returns to the streets of west London. 25-26 Aug. Feel the pomp at BBC Proms, an annual celebration of classical music at the Royal Albert Hall, which is in full swing in August. Events in London August 2024 16. Greenwich Music Time (August 6-10) Greenwich Music Time: Experience live performances at the historic Old Royal Naval College in Greenwich. The concert series features a mix of classical and contemporary acts. Contact Details +44 20 8269 4799. ... BBC Proms (All Month) The BBC Proms, the world's biggest and longest-running classical music festival, the BBC Proms is one of the top things 